In [1]:
%pip install semantic-link
import sempy
import sempy.fabric as fabric
import pandas as pd
import base64
import json
import requests
import fnmatch

# Init Fabric client

client = fabric.FabricRestClient()

StatementMeta(, 58c3f4e7-28d3-48f2-84a4-b42c55a0aa62, 8, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 69.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 121.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 125.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 88.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 156.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



# Get Item Definition Sample

In [28]:
workspaceId = fabric.get_workspace_id()
items = fabric.list_items(workspace=workspaceId)
itemId = items[items['Type'] == "Report"].iloc[0].Id

response = client.post(f"/v1/workspaces/{workspaceId}/items/{itemId}/getDefinition")

jsonParts = response.json()['definition']['parts']

df_parts = pd.json_normalize(jsonParts)

df_parts


StatementMeta(, 58c3f4e7-28d3-48f2-84a4-b42c55a0aa62, 36, Finished, Available)

,path,payload,payloadType
0,definition.pbir,ewogICJ2ZXJzaW9uIjogIjQuMCIsCiAgImRhdGFzZXRSZW...,InlineBase64
1,StaticResources/SharedResources/BaseThemes/CY2...,ewogICJuYW1lIjogIkNZMjNTVTA0IiwKICAiZGF0YUNvbG...,InlineBase64
2,StaticResources/RegisteredResources/Light44370...,ewogICIkc2NoZW1hIjogImh0dHBzOi8vcmF3LmdpdGh1Yn...,InlineBase64
3,StaticResources/RegisteredResources/_7abfc6c7-...,77+9UE5HDQoaCgAAAA1JSERSAAAAQAAAAEAIBgAAAO+/vW...,InlineBase64
4,report.json,ewogICJjb25maWciOiAie1widmVyc2lvblwiOlwiNS40M1...,InlineBase64


# Parameters

In [44]:
# Parameters

workspaceId = fabric.get_workspace_id()

#logoUrl = "https://github.com/microsoft/PowerBI-Icons/blob/main/PNG/Dataverse.png?raw=true"
logoUrl = "https://github.com/microsoft/PowerBI-Icons/blob/main/PNG/Desktop.png?raw=true"

# Download the new logo and convert to base 64

newContent = base64.b64encode(requests.get(logoUrl).content).decode('utf-8')

partsToReplace = ["*_7abfc6c7-1a23-4b5f-bd8b-8dc472366284171093267.jpg"] 

StatementMeta(, 58c3f4e7-28d3-48f2-84a4-b42c55a0aa62, 52, Finished, Available)

# Magic

In [45]:
# Get all items in the workspace

itemList = fabric.list_items(workspace=workspaceId)

# Filter only reports

itemList = itemList[itemList['Type'] == "Report"]

for index, item in itemList.iterrows():

    itemId = item['Id']

    print(f"Processing item '{itemId}'")

    # Get item definition parts

    response = client.post(f"/v1/workspaces/{workspaceId}/items/{itemId}/getDefinition")

    jsonParts = response.json()['definition']['parts']

    # Swap logo part

    partToReplace = [part for part in jsonParts 
                    if [res for res in partsToReplace 
                        if fnmatch.fnmatch(part["path"], res)]][0]

    if partToReplace is not None:

        partToReplace['payload'] = newContent

        # Update report definition with new logo

        payload = {}
        payload['definition'] =  {
            'parts' : jsonParts
        }

        response = client.post(f"/v1/workspaces/{workspaceId}/items/{itemId}/updateDefinition", json= payload)

    else:
        print("Cannot find any parts to replace in the report.")


StatementMeta(, 58c3f4e7-28d3-48f2-84a4-b42c55a0aa62, 53, Finished, Available)

Processing item '5bfcdb25-ac23-4e9a-93be-0399634c41dc'
Processing item '054e3ecf-56d4-4da5-9610-af4148fc839a'
Processing item '2efef62b-f80e-4679-9e17-7bac0c1fa178'
Processing item 'a55210be-1e8f-4929-a671-e49858d2fb56'
